In [1]:
import glob
import json
import os
import shutil

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

# local .json file with local path info
ungulates_root = os.path.dirname(os.getcwd())
local_paths_file = os.path.join(ungulates_root, 'local-paths.json')
with open(local_paths_file, "r") as json_file:
    local_paths = json.load(json_file)

import sys
sys.path.append(local_paths['overhead_functions_path'])

import koger_general_functions as kgf

In [2]:
# This is the observation name you are working on 
observation_name = 'observation088'

# These have to be manually gotten from the videos or frame extractions
# The first detectable rotation of the drone after "GoHome" is activated
# There is a excel file
go_home_frames = [8268, 12100, 9604]
# the clip number (within a flight) that go_home_frames are pulled from.
go_home_clips = [3, 3, 3]

In [3]:
# This is where the raw video images are saved 
frame_folders_root = os.path.join(local_paths['base_frames_folder'],
                                  observation_name)

In [4]:
videos_info_file = os.path.join(frame_folders_root, f"{observation_name}.csv")
videos_info = pd.read_csv(videos_info_file)


# Get first and last frame number for every video segment in each drone flight

first_last_folders = []

# current video name
curr_video_name = None

for row in range(videos_info.shape[0]):
    # sample the video at every other frame
    # csv records raw frame numbers (say 60fps)
    first_frame = int(videos_info.loc[row, 'first_frame'])
    last_frame = int(videos_info.loc[row, 'last_frame'])
    
    # If new video name add a new list to keep track of that videos info
    video_name = videos_info.loc[row, 'video_name'].rpartition('_')[0]
    if video_name != curr_video_name:
        first_last_folders.append([])
        curr_video_name = video_name
    if last_frame == -1:
        frames = glob.glob(
            os.path.join(frame_folders_root, videos_info.loc[row, 'video_name'], '*.jpg')
        )
        frames.sort(key=lambda file: int(file.split('_')[-1].split('.')[0]))
        last_frame = int(frames[-1].split('.')[-2].split('_')[-1]) + 2 # +2 because last frame is first frame after obs (and 60fps -> 30)
        
    first_last_folders[-1].append({'first_frame': first_frame, 'last_frame': last_frame})

In [5]:
flight_logs_folder = os.path.join(local_paths['base_data_path'], "flight_logs")
flight_log_files = glob.glob(
    os.path.join(flight_logs_folder, f"flightlog_ob{observation_name[-3:]}*")
)
flight_log_files.sort()

flight_logs = []
for file in flight_log_files:
    flight_log = pd.read_csv(file)
    # Get rid of potential white space in column names
    flight_log.rename(columns=lambda x: x.strip(), inplace=True)
    flight_logs.append(flight_log)

In [6]:
# In this part we estimate frame number at each row of drone log
home_frames =[]
for ind, log in enumerate(flight_logs):
    # first adjust the frame number to account for all the frames 
    # in previous clips for that flight.
    clip_ind = go_home_clips[ind]
    home_frame = go_home_frames[ind]
    for num in range(clip_ind):
        home_frame += int(first_last_folders[ind][num]['last_frame'])
    home_frames.append(home_frame)
    # go through the log and find the row where flycState == "GoHome" 
    # AND compass heading has changed >.2 since "GoHome" was activated. 
    # Treat this as the visually noted "going_home" frame
    go_home_rows = log['flycState'] == 'Go_Home'
    initial_heading = log[go_home_rows].iloc[0]['compass_heading(degrees)']
    rotation_started = abs(log['compass_heading(degrees)'] - initial_heading) >= 0.2
    going_home = log[go_home_rows & rotation_started].iloc[0]
    # Frame rate is 60fps so 16.66 milliseconds per frame.
    # Based on the the human frame label of the "go_home" frame we know what that
    # frame number should be. Use the linear relationship between frame number 
    # and time to estimate the frame number assosiated with every other frame.
    milli_per_frame = 16.666
    log.loc[:, 'frame_num'] = log.loc[:, 'time(millisecond)'] / milli_per_frame
    log.loc[:, 'frame_num'] -= going_home['time(millisecond)'] / milli_per_frame
    log.loc[:, 'frame_num'] = log.loc[:, 'frame_num'].astype(int)
    log.loc[:, 'frame_num'] += home_frames[ind] 

In [7]:
for ind, log in enumerate(flight_logs):
    log['altitude(m)'] = log['altitude(feet)'] / 3.28

In [8]:
# Get the first and last frame number for drone flight 
# (merging all video segments in drone flight together)
first_last_flights = []

for folder_list in first_last_folders:
    first_frame = folder_list[0]['first_frame']
    last_frame = folder_list[0]['last_frame']
    for fl_dict in folder_list[1:]:
            last_frame += fl_dict['last_frame']
        
    first_last_flights.append(
        {'first_frame': first_frame, 'last_frame': last_frame}
    )

In [9]:
# Go through every entry and record frames where drone has moved by a certrain
# threshold from previous saved frame.
# Use these frames to feed pix4d to create map of space and get ground truth drone movement
# Distance threshold to save another frame while drone is moving (degrees)
drone_movement_thresh = 0.00005
# How often to take frame when it is already rotating (degrees)
rotating_thresh = 30
# How many degrees the drone must turn for the drones to be considered rotating (degrees)
start_rotating_thresh = 1
# Speed theshold for moving (mph)
start_moving_thresh = 1.5
# Altitude threshold (meters)
altitude_thresh = 3

# List of lists to store the frame numbers being used in the map
frame_list = [[] for _ in range(len(flight_logs))]
# Record the log row index that coresponds to the each image in the map 
# Each flight has a different list
ind_list = [[] for _ in range(len(flight_logs))]
for flight_ind, log in enumerate(flight_logs):

    # index in flight log coresponding to first frame in recording
    recording_started = (log['frame_num'] 
                         >= first_last_flights[flight_ind]['first_frame'])
    first_ind = np.min(np.nonzero(recording_started.values))
    # index in flight log coresponding to last frame in recording
    recording_ended = (log['frame_num'] 
                       >= first_last_flights[flight_ind]['last_frame'])
    last_ind = np.min(np.nonzero(recording_ended.values))

    # 'observing' is true for row that correspond to frames used in the observation
    log['observing'] = False
    log.loc[first_ind:(last_ind-1), 'observing'] = True # -1 because pandas uses inclusive indexing
    # For images used in pix4d map, save the actual image name, none if not used in map
    log['image_name'] = None
    # Record the frames that are used to create the pix4d map
    log['used_in_map'] = False
    
    # Always include the first observation frame in the generated map
    ind_list[flight_ind].append(first_ind)
    frame_list[flight_ind].append(first_last_flights[flight_ind]['first_frame'])
    log.loc[first_ind, 'used_in_map'] = True

    # The coordinates of the last image added to the map
    last_coord = np.array([log.loc[first_ind, 'latitude'], 
                           log.loc[first_ind, 'longitude']])
    # The log row index of the last saved frame
    last_saved_row_ind = first_ind
    last_saved_heading = log.loc[first_ind, 'compass_heading(degrees)']
    # Last saved drone altitude
    last_saved_alt = log.loc[first_ind, 'altitude(m)']
    # The heading in the previous heading regardless of if saved
    last_seen_heading = log.loc[first_ind, 'compass_heading(degrees)']
    
    # Is the drone curently rotating
    rotating = False
    # Is the drone currently moving
    moving = False
    
    # For refernce
    # how many times a frame was added to the map to deal with rotation
    rotate_count = 0
    # How many times the drone was considered moving
    move_count = 0
    thresh_count = 0
    # How many times drone crossed altitude difference threshold
    altitude_count = 0

    for ind, row in log.loc[log['observing'] == True].iterrows():
        save_frame = False
        
        # How much has the drone rotated since the last saved frame
        heading_dif = np.abs(row['compass_heading(degrees)'] - last_saved_heading)
        if not rotating:
            if heading_dif > start_rotating_thresh:
                rotating = True
                save_frame = True
                rotate_count += 1
        elif rotating:
            if heading_dif > rotating_thresh:
                # The drone is still rotating
                save_frame = True
                rotate_count += 1
            elif row['compass_heading(degrees)'] - last_seen_heading == 0:
                # The drone has stopped rotating
                rotating = False
                save_frame = True
                rotate_count += 1
        # Needed to see when the drone has stopped rotating
        last_seen_heading = row['compass_heading(degrees)']
        # Starting to move
        if not moving and row['speed(mph)'] > start_moving_thresh:
            save_frame = True
            moving = True
            move_count += 1
        # Movement stopped
        if moving and row['speed(mph)'] == 0:
            save_frame = True
            moving = False
            move_count += 1

        cur_coord = np.array([row['latitude'], row['longitude']])
        # How much has the drone moved since the last saved frame
        diff = np.sqrt(np.sum(np.square(cur_coord - last_coord)))
        if diff > drone_movement_thresh:
            save_frame = True
            thresh_count += 1
            
        if np.abs(last_saved_alt - row['altitude(m)']) > altitude_thresh:
            save_frame = True
            altitude_count += 1

        if save_frame:
            frame_list[flight_ind].append(row['frame_num'])
            ind_list[flight_ind].append(ind)
            log.loc[ind, 'used_in_map'] = True
            last_coord = cur_coord
            last_saved_heading = row['compass_heading(degrees)']
            last_saved_alt = row['altitude(m)']


    # Add the last frame of the observation no matter what
    ind_list[flight_ind].append(last_ind-1)
    frame_list[flight_ind].append(first_last_flights[flight_ind]['last_frame']-2)
    log.loc[last_ind-1, 'used_in_map'] = True

    print(f"{log['used_in_map'].sum()}: {len(frame_list[flight_ind])} frames were added to the map from flight {flight_ind}.")
    print('{} frames were added from rotation'.format(rotate_count))
    print('{} frames were added from movement'.format(move_count))
    print('{} frames were added from continued movement'.format(thresh_count))
    print('{} frames were added from altitude movement'.format(altitude_count))
    

# print('{} frames were added to the map in total'.format(sum([len(l) for l in frame_list])))


34: 34 frames were added to the map from flight 0.
3 frames were added from rotation
18 frames were added from movement
11 frames were added from continued movement
0 frames were added from altitude movement
67: 67 frames were added to the map from flight 1.
14 frames were added from rotation
36 frames were added from movement
12 frames were added from continued movement
3 frames were added from altitude movement
91: 91 frames were added to the map from flight 2.
2 frames were added from rotation
28 frames were added from movement
57 frames were added from continued movement
3 frames were added from altitude movement


In [10]:
# Should the actual images be copied somewhere
save_images = True
save_modified_flight_logs = True
# Get the actual frames chosen above to construct map
# This is the place the map images will be stored
save_folder = os.path.join("/home/golden/Dropbox/map-params/worked_examples",
                           observation_name)
save_folder_images = os.path.join(save_folder, "map-images")
os.makedirs(save_folder_images, exist_ok=True)

all_folders = glob.glob(os.path.join(frame_folders_root, '*_*_*_*_*'))

video_folders = []
for folder in all_folders:
    # Get ride of some other folders in this folder
    if not (os.path.basename(folder).startswith('drone') 
            or os.path.basename(folder).startswith('observation')):
        video_folders.append(folder)
        
video_folders = [folder.rpartition('_')[0] for folder in video_folders]
# have only one entry for every video name
video_folders = list(set(video_folders))
video_folders.sort()

images_copied = 0
for video_ind, video_folder in enumerate(video_folders):
    # Folders for all the images extracted from the clips of a given flight
    image_folders = glob.glob(f"{video_folder}*")
    image_folders.sort()

    fl_dicts = first_last_folders[video_ind]
    # Frame_list contains the frame numbers that should be added to the map
    for map_frame_ind, frame_num in enumerate(frame_list[video_ind]):
        for folder_ind, fl_dict in enumerate(fl_dicts):
            if frame_num < fl_dict['last_frame']:
                # Frame is contained the this folder
                if frame_num % 2 != 0:
                    # make frame num even since those are the only frames that get extracted
                    frame_num -= 1
                image_file = glob.glob(
                    os.path.join(image_folders[folder_ind], f"*_*{frame_num:05d}.jpg")
                )[0]
                if not os.path.exists(image_file):
                    print('following file does not exist: {}'.format(image_file))
                else:
                    out_file = os.path.join(save_folder_images, 
                                            os.path.basename(image_file))
                    image_name = os.path.basename(image_file)
                    flight_logs[video_ind].loc[ind_list[video_ind][map_frame_ind], 'image_name'] = image_name
                    if save_images:
                        if images_copied % 50 == 0:
                            print(f"{images_copied} images copied.")
                        shutil.copyfile(image_file, out_file)
                        images_copied += 1
                break
            else:
                # frame number is in a later folder
                frame_num -= (fl_dict['last_frame'])
                if folder_ind >= len(fl_dicts) - 1:
                    print('failed', frame_list[video_ind][map_frame_ind])

# Save the dataframes in the output folder specified above
if save_modified_flight_logs:
    save_folder_logs = os.path.join(save_folder, 'drone-logs')
    os.makedirs(save_folder_logs, exist_ok=True)
    for flight_num in range(len(flight_logs)):
        filename = os.path.join(save_folder_logs, 
                                f"flight_{flight_num}.pkl")
        flight_logs[flight_num].to_pickle(filename)
        
        
    # Save the information about the frames used in map in .csv
    # saved to output folder specified above
    dfs = []
    for flight_ind in range(len(flight_logs)):
        dfs.append(flight_logs[flight_ind].loc[ind_list[flight_ind], ['image_name', 'latitude', 'longitude', 'altitude(m)']].copy())
    total_df = pd.concat(dfs, ignore_index=True)

    total_df.to_csv(os.path.join(save_folder, 'whole_obs_file_lat_long_alt.csv'), 
                    header=False, index=False)

print('This is where the images will be saved:', save_folder_images)

0 images copied.
50 images copied.
100 images copied.
150 images copied.
This is where the images will be saved: /home/golden/Dropbox/map-params/worked_examples/observation088/map-images
